In [2]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; float:center}</style>")

In [3]:
from __future__ import division, print_function

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import decomposition



%matplotlib inline

sns.set_context('notebook')
sns.set_style('ticks')

punctuation = ['.',',',':','!',';','-','?','"',"'",'(',')','—']   
other = ['ive','ve', "i've", "i'v", 'i’ll', 'i’ve', 'i’v']  # 'deb','hideb','don','didn','twaittry','doesn','thank','heydeb',
mystops = stopwords.words('english') + punctuation + other


import & sanity check

In [4]:
comments_with_sentences_100_classified = pd.read_csv('comments_with_sentences_100_classified_4classes.csv')

In [ ]:
comments_classified = pd.read_csv('comments_classified_SK_filtered2000_additional.csv',index_col=0)

comments_with_sentences_100 = pd.read_csv('comments_with_sentences_100.csv',index_col=0)
comments_only_100 = pd.read_csv('comments_only_100.csv',index_col=0)

comments_with_sentences_all = pd.read_csv('comments_with_sentences_all.csv',index_col=0)
comments_only_all = pd.read_csv('comments_only_all.csv',index_col=0)

In [ ]:
comments_with_sentences = comments_with_sentences.dropna()

comments_only_100 = comments_only.dropna()
comments_with_sentences_100 = comments_only.dropna()

comments_only_all = comments_only_all.dropna()
comments_with_sentences_all = comments_only_all.dropna()

In [ ]:
len(comments_only_all.title.unique())

In [ ]:
comments_with_sentences_all.columns

## process test & train data

### define data sets

In [5]:
comments_with_sentences_100_classified = comments_with_sentences_100_classified.dropna()
# X_train = X_train.dropna()

In [6]:
comments_classified.category.unique()
# comments_classified.category.replace('try','other',inplace=True)
# comments_classified.category.replace('addition','suggestion',inplace=True)
# comments_classified.category.replace('subtraction','substitution',inplace=True)
# comments_classified.category.replace('related','other',inplace=True)
# comments_classified.category.replace('question','other',inplace=True)

NameError: name 'comments_classified' is not defined

In [28]:
comments_with_sentences_100_classified.shape

(94611, 20)

In [ ]:
# encode classes labels as ints
# though this probably doesn't matter for this? 
le = LabelEncoder()
comments_classified.loc[:,'category_label'] = le.fit_transform(comments_classified.category)

list(le.classes_)

In [22]:
target.replace(['addition','substitution','omission'],'helpful',inplace=True)

In [21]:
X_train = comments_with_sentences_100_classified.sentence_tokens_stemmed[:50000] #\.as_matrix()
target = comments_with_sentences_100_classified.category[:50000] #.as_matrix()

X_test = comments_with_sentences_100_classified.loc[5000:, 'sentence_tokens_stemmed']
# X_test2 = comments_with_sentences_all.loc[:10000, 'sentence_tokens_stemmed']
# X_test = comments_with_sentences.loc[:, 'tokens_stemmed']

# X_train = comments_classified[comments_classified.category != 'other'].sentence #\.as_matrix()
# target = comments_classified[comments_classified.category != 'other'].category_label #.as_matrix()

## word relevancy - counter, tf-idf

In [23]:
vectorizer = CountVectorizer(ngram_range=(1,3), stop_words=mystops, max_df=50, min_df=5)

X_train_counts = vectorizer.fit_transform(X_train)
X_train_counts.shape

transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)

X_train_tf = transformer.transform(X_train_counts)
X_train_tf.shape

(50000, 9665)

just to try to get some sense of the frequencies

In [24]:
X_train_tf.toarray()[:,vectorizer.vocabulary_.get('suggest')].sum()

61439.640132201399

## classifier

train

In [25]:
classifier = MultinomialNB().fit(X_train_tf, target)

test

In [26]:
X_test_counts = vectorizer.transform(X_test)
X_test_tf = transformer.transform(X_test_counts)

In [27]:
predicted = classifier.predict(X_test_tf)

# for doc, category in zip(X_test, predicted):
# #     if category == 'addition':
#     if category != 'other':
#         print('%r => %s' % (doc, category))
        
print(len(predicted[predicted != 'other']))
print(len(predicted[predicted == 'helpful']))

# for doc, category in zip(X_test[:100], predicted[:100]):
#     print('%r => %s' % (doc, category))

1015
1015


In [30]:
for doc, category in zip(X_test[:3000], predicted[:3000]):
#     if category == 'addition':
    if category != 'other':
        print('%r => %s' % (doc, category))


'next time might zest shallot mix' => helpful
'seen ottolenghi method remov seed pomegran' => helpful
'recip chang' => helpful
'mint omit everyth els ad' => helpful
'use black cocoa powder recip' => helpful
'would suggest mayb fresh garlic instead' => helpful
'vegan baker friend make cake take stress day' => helpful
'substitu regular cocoa powder' => helpful
'chanc nip food shop local tesco pick larg tub green & black’ cocoa powder' => helpful
'hope enjoy trip sorri didn’t get one talk mayb next time' => helpful
'good tri per recip next time' => helpful
'use valrhona cocoa powder cake ghiradelli bake chocol chocol butter cream' => helpful
'cake flour work instead all-purpos' => helpful
'use dutch process cocoa go forward' => helpful
'thought decor somehow mayb instead vanilla buttercream think could fit fill etc' => helpful
'would like chocol buttercream frost instead vanilla' => helpful
'fyi saw strongli advis reduc sugar went ahead anyway 300g brown sugar instead 380g white sugar' =>

In [24]:
classifier = MultinomialNB().fit(X_test_tf, predicted)
# predicted = classifier.predict(X_test_tf)

In [ ]:
predicted[:10]

## PCA 

In [1]:
pca = decomposition.PCA(n_components=20)

pca.fit(X_train_tf.toarray())
X_trans = pca.transform(X_train_tf.toarray())

NameError: name 'decomposition' is not defined

In [ ]:
def encode_target(df, target_column):
    """Add column to df with integers for the target.

    Args
    ----
    df -- pandas DataFrame.
    target_column -- column to map to int, producing
                     new Target column.

    Returns
    -------
    df_mod -- modified DataFrame.
    targets -- list of target names.
    """
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod["Target"] = df_mod[target_column].replace(map_to_int)

    return (df_mod, targets)


In [ ]:
x,y = encode_target(comments_classified, 'category')

In [ ]:
comments_classified.loc[comments_classified.sentence.str.contains('instead of') == True, 'category'] = 'substitution'

# comments_classified

## Decision tree classifier

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree_classifier = DecisionTreeClassifier(max_depth=3, max_features=10)